In [1]:
import numpy as np
import pprint
import sys
if "../" not in sys.path:
  sys.path.append("../") 
from lib.envs.gridworld import GridworldEnv

In [2]:
pp = pprint.PrettyPrinter(indent=2)
env = GridworldEnv()

In [22]:
def policy_eval(policy, env, discount_factor=1.0, theta=0.00001):
    """
    Evaluate a policy given an environment and a full description of the environment's dynamics.
    
    Args:
        policy: [S, A] shaped matrix representing the policy.
        env: OpenAI env. env.P represents the transition probabilities of the environment.
            env.P[s][a] is a list of transition tuples (prob, next_state, reward, done).
            env.nS is a number of states in the environment. 
            env.nA is a number of actions in the environment.
        theta: We stop evaluation once our value function change is less than theta for all states.
        discount_factor: Gamma discount factor.
    
    Returns:
        V: vector of length env.nS representing the value function.
    """
    # Start with a random (all 0) value function
    V = np.zeros(env.nS)
    while True:
        # TODO: Implement!
        threshold = 0
        for s in range(env.nS):
            state_value = 0
            for a in range(env.nA): # 이 경우는 state 별 가능한 action이 전체집합으로 모두 동일하기 때문에 env.nA로 해도 됨. 그렇지 않은 경우는 enumerate(policy[s]) 로 사용.
                state_action_value = 0
                for i in env.P[s][a]: # i: s에서 a를 했을 때 이동 가능한 다음 state 정보
                    """
                    solution에서는 s'에 대한 summation을 하지 않음.
                    gridworld env에서는 action을 했을 때 s'이 deterministic하게 결정되기 때문에 괜찮지만,
                    다른 env에서도 적용하려면 아래처럼 s'에 대한 summation을 진행한 다음에 계산하는게 맞지 않을까?
                    """
                    state_action_value += (i[0] * V[i[1]]) # i[0]: transition prob , i[1]: next state

                state_value += (policy[s][a] * (env.P[s][a][0][2] + discount_factor * state_action_value))

            threshold = max(threshold, np.abs(V[s]-state_value))
            """
            위에서 계산한 threshold는 특정 s에 대한 threshold 임.
            따라서 iteration 종료 조건에 사용하기 위해서는 모든 s에 대한 threshold 중 최대값을 구해야 함.
            """
            V[s] = state_value
        
        if threshold < theta:
            break
    return np.array(V)

In [26]:
def policy_improvement(env, policy_eval_fn=policy_eval, discount_factor=1.0):
    """
    Policy Improvement Algorithm. Iteratively evaluates and improves a policy
    until an optimal policy is found.
    
    Args:
        env: The OpenAI envrionment.
        policy_eval_fn: Policy Evaluation function that takes 3 arguments:
            policy, env, discount_factor.
        discount_factor: gamma discount factor.
        
    Returns:
        A tuple (policy, V). 
        policy is the optimal policy, a matrix of shape [S, A] where each state s
        contains a valid probability distribution over actions.
        V is the value function for the optimal policy.
        
    """
    # Start with a random policy
    policy = np.ones([env.nS, env.nA]) / env.nA
    
    while True:
        # Implement this!
        v = policy_eval_fn(policy, env, discount_factor)
        policy_stable = True
        for s in range(env.nS):
            old_action = np.argmax(policy[s])
            action_value_list = []
            for action, action_prob in enumerate(policy[s]): # 각 action 별 value 를 구해서 그 중에 최대값을 가지는 action을 구하는 것이므로, action_prob는 사용할 필요가 없음.
                state_action_value = 0
                for prob, next_state, reward, done in env.P[s][action]:
                    state_action_value += prob * (reward + discount_factor * v[next_state])
                
                action_value_list.append(state_action_value)
            
            greedy_action = np.argmax(action_value_list)
            policy[s] = np.zeros(env.nA) # 현재 상태에서 optimal action이 아닌 action 들은 모두 고를 확률을 0으로 업데이트.
            policy[s][greedy_action] = 1 # optimal action 에 대해서만 확률을 1로 업데이트

            # policy distribution이 같을 때가 아니라, 모든 s에 대해 policy[s] update가 없을 때 iteration 종료. 
            if old_action != greedy_action: 
                policy_stable = False

        if policy_stable:                
            break
    
    return policy, v

In [27]:
policy, v = policy_improvement(env)
print("Policy Probability Distribution:")
print(policy)
print("")

print("Reshaped Grid Policy (0=up, 1=right, 2=down, 3=left):")
print(np.reshape(np.argmax(policy, axis=1), env.shape))
print("")

print("Value Function:")
print(v)
print("")

print("Reshaped Grid Value Function:")
print(v.reshape(env.shape))
print("")



Policy Probability Distribution:
[[1. 0. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]]

Reshaped Grid Policy (0=up, 1=right, 2=down, 3=left):
[[0 3 3 2]
 [0 0 0 2]
 [0 0 1 2]
 [0 1 1 0]]

Value Function:
[ 0. -1. -2. -3. -1. -2. -3. -2. -2. -3. -2. -1. -3. -2. -1.  0.]

Reshaped Grid Value Function:
[[ 0. -1. -2. -3.]
 [-1. -2. -3. -2.]
 [-2. -3. -2. -1.]
 [-3. -2. -1.  0.]]



In [29]:
# Test the value function
expected_v = np.array([ 0, -1, -2, -3, -1, -2, -3, -2, -2, -3, -2, -1, -3, -2, -1,  0])
np.testing.assert_array_almost_equal(v, expected_v, decimal=2)